In [2]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
import pandas as pd


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)'
                         ' Chrome/84.0.4147.135 Safari/537.36'}
vacancy = input('Введите вакансию: ')


def hh(vacancy):
    i = 0
    vacancy = vacancy.replace(' ', '+')
    jobs = []
    while True:
        html = f'https://spb.hh.ru/search/vacancy?L_is_autosearch=false&area=2&clusters=true&enable_snippets=true&search_field=name&text={vacancy}&page={i}'
        soup = bs(requests.get(html, headers=headers).text, 'html.parser')
        jobs_block = soup.find('div', {'class': 'vacancy-serp'})
        jobs_list = jobs_block.findChildren(recursive=False)
        for job in jobs_list:
            job_data = {}
            new_list = job.find('span', {'class': 'g-user-content'})
            if new_list != None:
                main_info = new_list.findChild()
                job_name = main_info.getText()
                job_link = main_info['href']
                salary = job.find('div', {'class': 'vacancy-serp-item__sidebar'})
                salary = salary.getText()
                salary = salary.replace(u'\xa0', u'')
                salaries = salary.split('-')
                salaries[0] = re.sub(r'[^0-9]', '', salaries[0])
                salary_min = salaries[0]
                if '' in salary_min:
                    salary_min = None
                if len(salaries) > 1:
                    salaries[1] = re.sub(r'[^0-9]', '', salaries[1])
                    salary_max = int(salaries[1])
                else:
                    salary_max = None
                job_data['vacancy'] = job_name
                job_data['salary_min'] = salary_min
                job_data['salary_max'] = salary_max
                job_data['link'] = job_link
                jobs.append(job_data)

        i += 1
        if not soup.find('a', attrs={'data-qa':'pager-next'}):
            break

    df = pd.DataFrame(jobs)
    return df

hh(vacancy)

Введите вакансию: бухгалтер


,vacancy,salary_min,salary_max,link
0,Бухгалтер (м.Обводный канал),None,40000.0,https://spb.hh.ru/vacancy/38641506?query=%D0%B...
1,Главный бухгалтер (заместитель главного бухгал...,None,80000.0,https://spb.hh.ru/vacancy/39182240?query=%D0%B...
2,Заместитель главного бухгалтера,None,NaN,https://spb.hh.ru/vacancy/39040529?query=%D0%B...
3,Главный бухгалтер,None,93600.0,https://spb.hh.ru/vacancy/39161394?query=%D0%B...
4,Помощник главного бухгалтера,None,40000.0,https://spb.hh.ru/vacancy/39163160?query=%D0%B...
...,...,...,...,...
1044,Ведущий бухгалтер отдела учета прочей хозяйств...,None,NaN,https://spb.hh.ru/vacancy/38177071?query=%D0%B...
1045,Ведущий бухгалтер / внутренний аудитор,None,90000.0,https://spb.hh.ru/vacancy/38645353?query=%D0%B...
1046,Ведущий бухгалтер на группу компаний,None,60000.0,https://spb.hh.ru/vacancy/38640707?query=%D0%B...
1047,Бухгалтер в сфере ЖКХ,None,NaN,https://spb.hh.ru/vacancy/38756182?query=%D0%B...
